In [ ]:
%%time
# coding: utf-8
!pip install -r requrements.txt
import nltk
import numpy as np
from scipy.sparse import csr_matrix, save_npz, load_npz, lil_matrix
from sklearn.preprocessing import normalize
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from collections import OrderedDict
#np.set_printoptions(threshold=np.inf)
import os

In [ ]:
%%time
stemmer = SnowballStemmer("russian")
extended_punctuation = sorted(set(punctuation).union(set(['–', '…', '«', '»', '*'])))

#text = 'Хотя сам в первый раз минут тридцать трясся, все не мог понять, этот водоворот черный или же в нем есть три желтые песчинки, что мерещатся мне в который уже раз... уф... Ну, готовься, сейчас мы нарушим табу Клана Мертвых Песков и несмотря на запрет войдем в руины Мирного... \nЯ поднял голову и вздрогнул – прямо надо мной нависал громадный красный череп, а вытекающая у него из клыкастого рта черная река песка казалось вот-вот накроет меня смертоносным душным валом... \nМы у самого Мирного... в десяти шагах от жестоко оплавленной стены. Давным-давно, когда Баронесса старательно драила котлы и помойные ведра, я как-то вернулась в свои покои и обнаружила, что моя постель и вещи изрезаны, измазаны грязью и отбросами, изразцовые стены покрыты черной краской.... Я в гневе выбежала наружу, искала тех, кто осмелился на такое,... и увидела ее – забившись в свой угол на кухонном дворе, эта замарашка пыталась отодрать от кожи остатки черной краски, она не подозревала, что это особый и крайне цепкий состав, что смывается лишь одним редким раствором.... Знал бы ты, что я с ней тогда сделала.... \n- Мне очень хочется узнать – я аж глаза закатил. Ощущая боль в локте и ступне, с головой наполненной злобными мыслями о затребованном Кирой королевском завтраке, не выспавшийся и недавно переживший кучу событий, как в реале, так и в Вальдире, я стоял перед соседкой заставившей меня изрядно потрепать нервы и на язык так и просилось «Да шли бы вы со своими пирожками, Варвара Павловна!» Но заглянув в глаза пошедшей на мировую женщины я осторожно принял тарелку и скомкано пробормотал:\n- Большое спасибо, Варвара Павловна, не стоило себя утруждать... спасибо... \nПовисла неловкая пауза, которую немедленно требовалось заполнить какими-нибудь умными словами, сказать что-то подходящее случаю, но в голове у меня плавал сонный туман внутри которого вертелась яичница с тремя целыми желтками.'
#text = 'Хотя сам в первый раз минут тридцать трясся, все не мог понять, этот водоворот черный или же в нем есть три желтые песчинки, что мерещатся мне в который уже раз... уф...' *2
#text = 'a b c b a a'
corpusdir = os.path.expanduser("~") + '/book/'
files = os.listdir(corpusdir)
text_dict = {}
str_punkt = ''.join(extended_punctuation)
str_empty = ' ' * len(str_punkt)

def cleanup(txt):
    table = str.maketrans(str_punkt, str_empty)
    txt = txt.translate(table).lower()
    table_2 = str.maketrans('qwertyuiopasdfghjklzxcvbnm0123456789', str_empty)
    txt = txt.translate(table_2)
    return txt


for file in files:
    fullpath =  corpusdir + file
    print("reading from {}".format(fullpath))
    text = open( fullpath, 'r').read()
    text_dict[file] = cleanup(text)

In [ ]:
%%time
word_dict = {}
for k, v in text_dict.items():
    word_dict[k] = [stemmer.stem(word) for word in nltk.word_tokenize(v)]

In [ ]:
%%time
for k, v in word_dict.items():
    print(k)
    print(' '.join(v[:10]))

In [ ]:
%%time
#text.replace('-', ' ')
#words = [stemmer.stem(word) for word in nltk.word_tokenize(text.replace('-', ' ').lower()) if word not in extended_punctuation]
words = []
for i in word_dict.values():
    words = words + i
print(len(words))

In [ ]:
%%time
lemlist = sorted([x for x in (set(words))])
order = len(lemlist)
print(len(words), order)

In [ ]:
%%time
#M = np.zeros((order,order))
#M = csr_matrix((order, order))
M = lil_matrix((order, order))
print(type(M))
print(M.shape)
print(M[1,2])

In [ ]:
%%time
for n, m in word_dict.items():
    print("Working on " + n)
    for i, j in zip(m[:-1],m[1:]):
        x = lemlist.index(i)
        y = lemlist.index(j)
        #print(i,j,x,y)
        M[x, y] = M[x, y] + 1
#print("  " + " ".join(words))
#print("  " + " ".join(lemlist))
#print(M.shape)

In [ ]:
%%time
M_normalized = normalize(M, norm='l1', axis=1, copy=False)
#M.todence()

In [ ]:
%%time
save_npz('book_mm.npz', M_normalized)

In [ ]:
%%time
open('lemlist.txt', 'w').write('\n'.join(lemlist))